In [1]:
import os
import pandas as pd
from git import Repo

In [2]:
#клонируем репозиторий на диск
Repo.clone_from('https://github.com/andreamust/MusicMetaLinker.git', 
                to_path = 'D:/programs/projects/yandex_music/MusicMetaLinker')
#меняем директорию
os.chdir('D:/programs/projects/yandex_music/MusicMetaLinker')
#устанавливаем
!pip install -r requirements.txt

In [3]:
pip install deezer-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
import MusicMetaLinker
from MusicMetaLinker.linking import linking

In [5]:
meta = pd.read_json('D:/programs/projects/yandex_music/meta.json', lines= True, convert_dates=['dttm'])
meta.info()
display(meta.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71769 entries, 0 to 71768
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   track_id  71768 non-null  object        
 1   dttm      71768 non-null  datetime64[ns]
 2   title     71768 non-null  object        
 3   language  21969 non-null  object        
 4   isrc      71455 non-null  object        
 5   genres    71768 non-null  object        
 6   duration  71768 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 3.8+ MB


,track_id,dttm,title,language,isrc,genres,duration
29920,a464bab3f04e0e72e1a2636c7e8d646d,2021-08-09 18:28:57,My Life Is Going On,None,QZMZ92194649,[POP],225480.0
3963,fea613a7958a7f771381dd36150d725e,2022-01-21 21:44:20,God Bless the USA,EN,QZPJ32137881,[POP],185920.0
50283,cf305a65c3ac08bb649f21f8fcf102e6,2013-06-24 22:00:49,Get Me Out of Here,None,USHM21264440,"[METAL, INDUSTRIAL]",285340.0
10513,7ad13a229b0a28adb83630b8bb9f5580,2022-05-16 22:03:18,Atrasadinha,None,QZHZ62230535,"[FOLK, LATINFOLK]",148360.0
6888,82af6e159ef993dd321d38b1cf7907a0,2022-03-02 22:18:49,abcdefu,None,QZGLS2242864,[ELECTRONICS],175380.0


In [6]:
#Оригинальный скрипт
from MusicMetaLinker.linking import linking

# Create a linking object
linker = linking.Align(
                artist="The Beatles",
                album="Let It Be",
                track="Let It Be",
                track_number=4,
                duration=168.25,
#                release_year=1970,
#                musicbrainz_id="b10bbbfc-cf9e-42e0-be17-e2c3e1d2600d",
                isrc="USAR10000048",
                strict=False,
            )
# Link the metadata
track_name = linker.get_track()
artist_name = linker.get_artist()
album_name = linker.get_album()
track_number = linker.get_track_number()
duration = linker.get_duration()
release_year = linker.get_release_date()
isrc = linker.get_isrc()

# Get the identifiers
links = {'musicbrainz': linker.get_mbid(),
         'isrc': linker.get_isrc() if isrc is None else isrc,
         'deezer': linker.get_deezer_id(),
         'deezer_url': linker.get_deezer_link(),
         'youtube_url': linker.get_youtube_link(),
         }

Searching for The Beatles - Let It Be on YouTube Music


In [7]:
#наш сбор данных по всем isrc в meta
#import MusicMetaLinker
from MusicMetaLinker.linking import linking
for i in range(0,5):#range(len(meta)):
    linker = linking.Align(
                isrc=meta.loc[i,'isrc'],
                strict=False,
            )
    # Link the metadata
    #track_name = linker.get_track()
    #artist_name = linker.get_artist()
    #album_name = linker.get_album()
    #track_number = linker.get_track_number()
    #duration = linker.get_duration()
    #release_year = linker.get_release_date()
    isrc = linker.get_isrc()
    meta.loc[i, 'linker_release_year'] = linker.get_release_date()
    meta.loc[i, 'linker_artist_name'] = linker.get_artist()
    meta.loc[i, 'linker_track_name'] = linker.get_track()
    meta.loc[i, 'linker_album_name'] =  linker.get_album()
    # Get the identifiers
    links = {#'musicbrainz': linker.get_mbid(),
             'isrc': linker.get_isrc() if isrc is None else isrc,
             #'deezer': linker.get_deezer_id(),
             #'deezer_url': linker.get_deezer_link(),
             #'youtube_url': linker.get_youtube_link(),
             }
display(meta.head(10))
display(meta.sample(20))


Searching for None - None on YouTube Music
Searching for None - None on YouTube Music
Searching for None - None on YouTube Music
Searching for None - None on YouTube Music
Searching for None - None on YouTube Music


,track_id,dttm,title,language,isrc,genres,duration,linker_release_year,linker_artist_name,linker_track_name,linker_album_name
0,c3b9d6a354ca008aa4518329aaa21380,2021-12-16 21:00:00,Happy New Year,EN,RUB422103970,[DANCE],161120.0,2021,Oliver Tree,Happy New Year,"Ugly is Beautiful: Shorter, Thicker & Uglier (..."
1,c57e3d13bbbf5322584a7e92e6f1f7ff,2021-11-24 14:00:33,Bad Habits,EN,QZN882178276,[ELECTRONICS],362260.0,2021,Oliver Tree,Bad Habits,"Ugly is Beautiful: Shorter, Thicker & Uglier (..."
2,955f2aafe8717908c140bf122ba4172d,2021-11-24 15:37:03,Por Esa Loca Vanidad,None,QZNJZ2122549,"[FOLK, LATINFOLK]",260000.0,2021,Black,Por Esa Loca Vanidad,Comedy
3,fae5a077c9956045955dde02143bd8ff,2021-11-24 15:37:03,Mil Lagrimas,None,QZNJZ2166033,"[FOLK, LATINFOLK]",190000.0,2021,Black,Mil Lagrimas,Comedy
4,6bede082154d34fc18d9a6744bc95bf5,2021-11-24 15:37:03,Sexo Humo y Alcohol,None,QZNJZ2122551,"[FOLK, LATINFOLK]",203000.0,2021,Black,Sexo Humo y Alcohol,Comedy
5,551331ee1444de6658e71933eb7f57a2,2021-11-24 15:37:03,La Ultima Vez,None,QZNJZ2166034,"[FOLK, LATINFOLK]",173000.0,NaN,NaN,NaN,NaN
6,2be8bb01bb1aeef0908524dc11935a88,2021-11-24 15:37:03,Homenaje a Pastor de los Santos,None,QZNJZ2122553,"[FOLK, LATINFOLK]",192100.0,NaN,NaN,NaN,NaN
7,3883d1add3ff8b60e77548a64df7c892,2021-11-24 15:37:03,Los Marginales,None,QZNJZ2122554,"[FOLK, LATINFOLK]",311500.0,NaN,NaN,NaN,NaN
8,59aa886979699aa7e4e2a8aed43bfe03,2021-11-24 15:37:03,Cuenta Conmigo,None,QZNJZ2122555,"[FOLK, LATINFOLK]",250000.0,NaN,NaN,NaN,NaN
9,2a38992b7769963937246288ad89b44c,2021-11-24 15:37:03,Ahora Juego Yo,None,QZNJZ2122556,"[FOLK, LATINFOLK]",156000.0,NaN,NaN,NaN,NaN


,track_id,dttm,title,language,isrc,genres,duration,linker_release_year,linker_artist_name,linker_track_name,linker_album_name
70526,3750089167196beac06e1694c2939e12,2019-04-09 13:31:43,Love Really Hurts Without You,None,PLS921753414,[],187750.0,NaN,NaN,NaN,NaN
44412,d0dfd363ad98fd92e2df4a84f2af66a3,2015-12-18 04:48:36,Perfect,None,TCACK1578697,[ALTERNATIVE],203620.0,NaN,NaN,NaN,NaN
11379,812117f66ae225eb7c3edd5a0b7c21c6,2022-06-06 16:48:29,Sabor a Mi,None,QZMZ92246811,"[FOLK, LATINFOLK]",153750.0,NaN,NaN,NaN,NaN
12034,36467327ce0f8fd36344dd97877a3b59,2022-06-21 21:56:59,Tchuco Nela,None,QZNJV2248843,"[FOLK, LATINFOLK]",158030.0,NaN,NaN,NaN,NaN
46660,6ebe4d1ab18e9eca2b16747f48eeb46c,2011-11-08 20:05:39,Por Arriba Por Abajo,None,USA370634459,"[FOLK, LATINFOLK]",185920.0,NaN,NaN,NaN,NaN
35655,6290721e6bd294417fe8d0eadd45dfca,2021-10-01 01:25:13,Chega e Senta,None,QZNJX2104328,"[FOLK, LATINFOLK]",155340.0,NaN,NaN,NaN,NaN
23670,56b2dc717000a8ad63f9f015cadfde1d,2021-05-29 14:58:21,Você Não Vai Parar - Instrumental,None,QZHZ52122683,[SPIRITUAL],223840.0,NaN,NaN,NaN,NaN
67128,e1d4c9ee7912dbfde9dcd1e8ff16f737,2018-10-22 21:00:00,One Kiss,EN,CH6541877991,[DANCE],222250.0,NaN,NaN,NaN,NaN
14046,09c7068f86386e94255ea18417169d02,2022-08-01 20:15:49,Áudio Que Te Entrega,None,QZNJY2212675,"[FOLK, LATINFOLK]",171500.0,NaN,NaN,NaN,NaN
31842,8c31b6eb7f875096cdc7a06edd58f23b,2021-08-27 23:19:38,Bezos I + II,EN,QZGWX2220513,[DANCE],274240.0,NaN,NaN,NaN,NaN
